In [ ]:
#|default_exp concurrent.remote.test_ProxyEvent_0

In [ ]:
#|export
import asyncio, os, sys, subprocess
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager, find_available_port
from fbdev.dev_utils import is_in_repl
from fbdev.concurrent.remote import RemoteController, get_client, ProxyEvent, ProxyEventMediator
import random

In [ ]:
#|export
port_num = find_available_port()

remote_py_script = """
address = ('localhost', %s)
authkey = b'secret'

import asyncio, os, multiprocessing
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager
from fbdev.concurrent.remote import RemoteController, ProxyEvent, ProxyEventMediator

listener = Listener(address, authkey=authkey)
async def main(conn):
    task_manager = TaskManager(None)
    remote = RemoteController(conn, task_manager)
    
    my_local_event1 = asyncio.Event()
    proxy_event_mediator1 = ProxyEventMediator('my_event1', remote, task_manager, my_local_event1)
    proxy_event2 = ProxyEvent('my_event2', remote)
    
    remote.send_ready()
    await remote.await_ready()
    
    await asyncio.sleep(1)
    
    print('Child: setting `my_local_event1`')
    my_local_event1.set()
    
    await asyncio.sleep(0.5)
    
    print('Child: setting `proxy_event2`')
    proxy_event2.set()
    
    await asyncio.sleep(1)
    
with listener.accept() as conn:
    asyncio.run(main(conn))
""" % port_num

address = ('localhost', port_num)
authkey = b'secret'

async def main():
    task_manager = TaskManager(None)
    proc = subprocess.Popen([sys.executable, '-c', remote_py_script])

    with get_client(address, authkey=authkey) as conn:
        remote = RemoteController(conn, task_manager)
        
        proxy_event1 = ProxyEvent('my_event1', remote)
        my_local_event2 = asyncio.Event()
        proxy_event_mediator2 = ProxyEventMediator('my_event2', remote, task_manager, my_local_event2)
        
        remote.send_ready()
        await remote.await_ready()
        
        print('Parent: Awaiting `proxy_event1`')
        await proxy_event1.wait()
        print('Parent: Awaited `proxy_event1`')
        
        print('Parent: Awaiting `my_local_event2`')
        await my_local_event2.wait()
        print('Parent: Awaited `my_local_event2`')
        
        await asyncio.sleep(1)
        
    await task_manager.destroy()
    stdout, stderr = proc.communicate()  # This will wait until the process completes
    
if not is_in_repl():
    asyncio.run(main())

In [ ]:
await main()

Parent: Awaiting `proxy_event1`
Child: setting `my_local_event1`
Parent: Awaited `proxy_event1`
Parent: Awaiting `my_local_event2`
Child: setting `proxy_event2`
Parent: Awaited `my_local_event2`
